## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-04-08-23 +0000,nyt,United Airlines Resumes Operations After Syste...,https://www.nytimes.com/2025/08/06/business/un...
1,2025-08-07-04-05-25 +0000,bbc,Trump's new tariffs take effect in trade war e...,https://www.bbc.com/news/articles/cx23jmvn5yzo...
2,2025-08-07-04-03-19 +0000,nyt,Staggering U.S. Tariffs Begin as Trump Widens ...,https://www.nytimes.com/2025/08/07/business/ec...
3,2025-08-07-04-02-37 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/06/busine...
4,2025-08-07-04-00-00 +0000,wapo,"Trump’s tariffs take effect, potentially upend...",https://www.washingtonpost.com/business/2025/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,59
179,india,16
7,new,14
8,tariffs,13
391,russia,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
203,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,126
134,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,115
220,2025-08-06-14-14-46 +0000,nypost,Trump slaps additional 25% tariff on India ove...,https://nypost.com/2025/08/06/us-news/trump-sl...,109
211,2025-08-06-14-50-35 +0000,nypost,Apple commits another $100B to US manufacturin...,https://nypost.com/2025/08/06/business/apple-c...,108
95,2025-08-06-21-56-54 +0000,wapo,"Trump raises tariffs on India to 50 percent, c...",https://www.washingtonpost.com/business/2025/0...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
203,126,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
134,56,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
47,46,2025-08-07-00-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
274,42,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
43,40,2025-08-07-00-10-12 +0000,nypost,Out-of-control driver accused of killing 4 Pep...,https://nypost.com/2025/08/06/us-news/fraser-m...
34,38,2025-08-07-01-00-00 +0000,wsj,Economic activity tied to manufacturing has sh...,https://www.wsj.com/economy/trump-pledged-to-b...
175,38,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
70,34,2025-08-06-22-55-26 +0000,nypost,NYC designer Martha Nolan was an Irish immigra...,https://nypost.com/2025/08/06/us-news/nyc-desi...
26,33,2025-08-07-01-23-27 +0000,wapo,"One dead, 13 injured in France wildfire spanni...",https://www.washingtonpost.com/world/2025/08/0...
212,30,2025-08-06-14-46-29 +0000,bbc,US journalist found alive days after going mis...,https://www.bbc.com/news/articles/cn72x513v8ko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
